# OMPython Example

OMPython is the Python library that allows you to run Open Modelica simulations and to get results back within a Python program.

Documentation for the library is here:

https://openmodelica.org/doc/OpenModelicaUsersGuide/latest/ompython.html

The following example uses the *enhanced* OMPython API - for this you create a `ModelicaSystem` object as follows:

In [1]:
from OMPython import ModelicaSystem
mod=ModelicaSystem("C:/Notebooks/ENG4350/SatModel.mo","SatModel.Satellite", "Modelica.Constants")

2018-03-09 14:30:45,300 - OMPython - INFO - OMC Server is up and running at file:///C:/Users/Hugh/AppData/Local/Temp/openmodelica.port.9d56ade445f74d64818ca3d4cdcd830d


Expected end of text (at char 1119), (line:7, col:28)


...looks like some sort of coding issue in the Modelica.Constants library...

Note the following in the command 
1. The path uses the forward slashes `/` to specify where the `mo` file is (not the standard Windows back slashes `\`).  
2. The model imports the value of $\pi$ from the `Modelica.Constants` library, so this is included in the function call.

Now that we have our `ModelicaSystem` object we can set the parameters for the Satellite model.  For the `Satellite` OpenModelica model, the paramters are the Mean Anomaly at Epoch(`M0`), Mean Motion at Epoch (`N0`), eccentricity (`ecc`), 1st time derivative of Mean Motion divided by 2 (`Ndot2`) and 2nd derivative of Mean Motion divided by 6 (`Nddot6`).

So we set these parameters, then we make sure we set them properly

In [2]:
mod.setParameters(M0=267.1230, N0=2.005651, ecc=0.0034407, Ndot2=0.00000068, Nddot6=0.)
mod.getParameters()

{'M0': 267.123,
 'N0': 2.005651,
 'Nddot6': 0.0,
 'Ndot2': 6.8e-07,
 'ecc': 0.0034407}

Next we can set the simulation options.  It appears that we cannot set the stopTime AND the startTime at the same time!  Try this yourself...

In [4]:
mod.setSimulationOptions(stopTime=88200., stepSize=60.)
mod.setSimulationOptions(startTime=86400.)
mod.getSimulationOptions()

{'solver': 'dassl',
 'startTime': 86400.0,
 'stepSize': 60.0,
 'stopTime': 88200.0,
 'tolerance': 1e-06}

Next we perform the simulation and get the outputs

In [5]:
mod.simulate()
mod.getSolutions()

('E',
 'M',
 'M0',
 'N0',
 'Nddot6',
 'Ndot2',
 'a',
 'der($STATESET1.x)',
 'der(E)',
 'der(M)',
 'der(p_sat_pf.x)',
 'der(p_sat_pf.y)',
 'der(r)',
 'der(theta)',
 'ecc',
 'p_sat_pf.x',
 'p_sat_pf.y',
 'p_sat_pf.z',
 'r',
 'theta',
 'time',
 'v_sat_pf.x',
 'v_sat_pf.y',
 'v_sat_pf.z')

We can now get the results as numpy arrays by doing the following:

In [7]:
(time, M, E, theta) = mod.getSolutions("time", "M", "E", "theta")

The data is now in a form that we can use Python to work with.  For example we can create an interactive plot quite easily

In [8]:
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

In [9]:
p = figure(width=900, height=700, 
           title="Angle vs. Time",
          tools="pan,wheel_zoom,box_zoom,reset,crosshair, save")

p.xaxis.axis_label = 'Time (EpSec)'
p.yaxis.axis_label = 'Angle (deg)'
p.background_fill_color = "khaki"

p.xgrid.grid_line_color = 'black'
p.xgrid[0].grid_line_alpha=0.8
p.xgrid.minor_grid_line_color = 'black'
p.xgrid.minor_grid_line_alpha = 0.2

p.ygrid.grid_line_color = 'black'
p.ygrid[0].grid_line_alpha=0.8
p.ygrid.minor_grid_line_color = 'black'
p.ygrid.minor_grid_line_alpha = 0.25

l1 = p.line(x=time, y=M, color='red', line_width=2, legend='M, Mean Anomaly')
p.add_tools(HoverTool(renderers=[l1], tooltips=[("(t,M)", "($x, $y)"),]))

l2 = p.line(x=time, y=E, color='green', line_width=2, legend='E, Eccentric Anomaly')
p.add_tools(HoverTool(renderers=[l2], tooltips=[("(t,E)", "($x, $y)"),]))

l3 = p.line(x=time, y=theta, color='blue', line_width=2, legend='θ, True Anomaly')
p.add_tools(HoverTool(renderers=[l3], tooltips=[("(t,theta)", "($x, $y)"),]))

show(p)

To see all of the commands we have, we can do a directory of the `mod` object:

In [10]:
dir(mod)

['_ModelicaSystem__checkAvailability',
 '_ModelicaSystem__checkTuple',
 '_ModelicaSystem__createQuantitiesList',
 '_ModelicaSystem__getContinuousNames',
 '_ModelicaSystem__getContinuousValues',
 '_ModelicaSystem__getInputNames',
 '_ModelicaSystem__getInputValues',
 '_ModelicaSystem__getMatrix',
 '_ModelicaSystem__getMatrixA',
 '_ModelicaSystem__getMatrixB',
 '_ModelicaSystem__getMatrixC',
 '_ModelicaSystem__getMatrixD',
 '_ModelicaSystem__getOutputNames',
 '_ModelicaSystem__getOutputValues',
 '_ModelicaSystem__getParameterNames',
 '_ModelicaSystem__getParameterValues',
 '_ModelicaSystem__getQuantitiesNames',
 '_ModelicaSystem__getSimulationValues',
 '_ModelicaSystem__getXXXs',
 '_ModelicaSystem__loadingModel',
 '_ModelicaSystem__setInputSize',
 '_ModelicaSystem__setOptions',
 '_ModelicaSystem__setValue',
 '_ModelicaSystem__simInput',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__ha

As you put together your code, you will need to be able to instantiate more than one Satellite object in your model.  You will also have to use the ModelicaSystem object to set parameters of each individual object.  An example of this is below.

In [11]:
gps=ModelicaSystem("C:/Notebooks/ENG4350/SatModel.mo","SatModel.GPS", "Modelica.Constants")

2018-03-09 14:32:12,433 - OMPython - INFO - OMC Server is up and running at file:///C:/Users/Hugh/AppData/Local/Temp/openmodelica.port.270019bde1864ed297d0083d79b46e51


Expected end of text (at char 2183), (line:12, col:34)


In [12]:
gps.setParameters(**{'prn13.M0':267.1230, 'prn13.N0':2.005651, 'prn13.ecc':0.0034407, 'prn13.Ndot2':0.00000068, 
                     'prn13.Nddot6':0.})
gps.setParameters(**{'prn11.M0':280.1424, 'prn11.N0':2.005527, 'prn11.ecc':0.0164899, 'prn11.Ndot2':-0.00000021, 
                     'prn11.Nddot6':0.})
gps.getParameters()

{'prn11.M0': 280.1424,
 'prn11.N0': 2.005527,
 'prn11.Nddot6': 0.0,
 'prn11.Ndot2': -2.1e-07,
 'prn11.ecc': 0.0164899,
 'prn13.M0': 267.123,
 'prn13.N0': 2.005651,
 'prn13.Nddot6': 0.0,
 'prn13.Ndot2': 6.8e-07,
 'prn13.ecc': 0.0034407}

It appears that each time you create a `ModelicaSystem` object, you create another connection to OpenModelica.  Python handles this by starting another Windows process.  If you have more than about 5 of these going, Windows complains.  So to shutdown this object we do a `del` in Python

In [13]:
del mod

In [14]:
mod.getParameters()

NameError: name 'mod' is not defined

The above `getParameters()` generates a Python error...